In [2]:
import xarray as xr
import glob
import numpy as np
import geopandas as gpd
import pandas as pd
import sys
sys.path.append('../')
from utilities.era5_down import nc_inter_2D, add_variable_along_timelatlon, annual_smb_glacier
from utilities.plot_results import plot_smb

from dask.distributed import Client
client = Client()

In [3]:
def run_dask():
    input_dem = '../data/static/SSI_static_2000_1000.nc'
    ds_dem = xr.open_dataset(input_dem)

    files = sorted(glob.glob('../data/ERA_59_20_day/*.nc'))

    ds = xr.open_dataset(files[0])
    dso = nc_inter_2D(ds, ds_dem, 'quadratic')
    print('Done:'+' '+ files[0][22:])

    for i in range(len(files))[1:]:
        ds = xr.open_dataset(files[i])
        dso1 = nc_inter_2D(ds, ds_dem, 'quadratic')
        dso = xr.merge([dso, dso1])
        #print(dso)
        print('Done:'+' '+ files[i][22:])

    dso  = dso.where(dso['MASK']==1, np.nan) #[['T2','PDD','SF']]
    SMB  = dso['SF'] + (dso['PDD'] * -0.0054)
    MO   = ((0.003 * dso['T2'] + 0.52)* dso['SF'])
    MELT = (dso['PDD'] * -0.0084)
    RF   = MELT + MO

    add_variable_along_timelatlon(dso, MELT.values, 'MELT', 'm of water equivalent', 'Melt')
    add_variable_along_timelatlon(dso, SMB.values, 'SMB', 'm of water equivalent', 'Surface Mass Banlance')
    add_variable_along_timelatlon(dso, RF.values, 'Q', 'm of water equivalent', 'Runoff')
    add_variable_along_timelatlon(dso, MO.values, 'RZ', 'm of water equivalent', 'Refreezing')

    #dso.to_netcdf('../data/ERA_59_20_day_int/int_era5_1959_2020.nc')
    #breakpoint()

    df_g = gpd.read_file('../data/static/Shapefiles/SSI_all_fff.shp')
    df_smb = pd.read_csv('../data/mass_balance/SSI_SMB.csv', sep='\t', index_col=['YEAR'])

    glaciers_shp = ['Hurd Glacier', 'Johnsons Glacier', 'Belling Glacier']
    glaciers_smb = ['HURD', 'JOHNSONS', 'BELLINGSHAUSEN']

    for i in range(len(glaciers_shp)):
        df_all = annual_smb_glacier(df_g, glaciers_shp[i], dso, df_smb, glaciers_smb[i])
        fig = plot_smb(df_all)
        # Finalmente guardamos nuestra figura
        print('Done:'+' '+ glaciers_smb[i])
        fig.savefig('../fig/'+ glaciers_smb[i] +'.png', dpi = 200, facecolor='w', bbox_inches = 'tight', 
                    pad_inches = 0.1)


In [4]:
client.submit(run_dask)

<Future: pending, key: run_dask-e3267defa08b150f8a95cadccf43062d>

Done: era5_1959_day.nc
Done: era5_1960_day.nc
Done: era5_1961_day.nc
Done: era5_1962_day.nc
Done: era5_1963_day.nc
Done: era5_1964_day.nc
Done: era5_1965_day.nc
Done: era5_1966_day.nc
Done: era5_1967_day.nc
Done: era5_1968_day.nc
Done: era5_1969_day.nc
Done: era5_1970_day.nc
Done: era5_1971_day.nc
Done: era5_1972_day.nc
Done: era5_1973_day.nc
Done: era5_1974_day.nc
Done: era5_1975_day.nc
Done: era5_1976_day.nc
Done: era5_1977_day.nc
Done: era5_1978_day.nc
Done: era5_1979_day.nc
Done: era5_1980_day.nc
Done: era5_1981_day.nc
Done: era5_1982_day.nc
Done: era5_1983_day.nc
Done: era5_1984_day.nc
Done: era5_1985_day.nc
Done: era5_1986_day.nc
Done: era5_1987_day.nc
Done: era5_1988_day.nc
Done: era5_1989_day.nc
Done: era5_1990_day.nc
Done: era5_1991_day.nc
Done: era5_1992_day.nc
Done: era5_1993_day.nc
Done: era5_1994_day.nc
Done: era5_1995_day.nc
Done: era5_1996_day.nc
Done: era5_1997_day.nc
Done: era5_1998_day.nc
Done: era5_1999_day.nc
Done: era5_2000_day.nc
Done: era5_2001_day.nc
Done: era5_

2022-11-18 12:01:25,154 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.40 GiB -- Worker memory limit: 1.92 GiB


Done: era5_2003_day.nc


2022-11-18 12:01:27,126 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.39 GiB -- Worker memory limit: 1.92 GiB


...

Done: era5_2013_day.nc


2022-11-18 12:01:46,828 - distributed.worker_memory - WARNING - Worker is at 31% memory usage. Resuming worker. Process memory: 619.12 MiB -- Worker memory limit: 1.92 GiB
2022-11-18 12:01:48,428 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.37 GiB -- Worker memory limit: 1.92 GiB
2022-11-18 12:01:48,626 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.44 GiB -- Worker memory limit: 1.92 GiB
2022-11-18 12:01:48,728 - distributed.worker_memory - WARNING - Worker is at 81% memory usage. Pausing worker.  Process me

Done: era5_2014_day.nc


2022-11-18 12:01:50,825 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.46 GiB -- Worker memory limit: 1.92 GiB
2022-11-18 12:01:50,928 - distributed.worker_memory - WARNING - Worker is at 82% memory usage. Pausing worker.  Process memory: 1.57 GiB -- Worker memory limit: 1.92 GiB
2022-11-18 12:01:50,928 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.57 GiB -- Worker memory limit: 1.92 GiB
2022-11-18 12:01:51,026 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory lea

Done: era5_2015_day.nc
